In [1]:
wd <- dirname(dirname(getwd()))
source(paste0(wd,"/mission_control/treasure_map.R"))
source(paste0(wd,"/mission_control/helpers/xgboost/eval_help.R"))
source(paste0(wd,"/mission_control/helpers/xgboost/paper_settings.R"))
source(paste0(wd,"/mission_control/helpers/xgboost/validation_help.R"))
library(tidyverse)

Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.1
Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘tidyr’ was built under R version 3.6.3”Warning message:
“package ‘purrr’ was built under R version 3.6.3”Warning message:
“package ‘dplyr’ was built under R version 3.6.3”Warning message:
“package ‘forcats’ was built under R version 3.6.3”── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

### 0 - Read in external studies and bind rows

In [2]:
validation_ready <- readRDS( paste0( TMP_DIR, "validation-go.Rds"  ))
hmf_models <- readRDS( paste0( TMP_DIR, "validation-hmf-models.Rds"))

In [3]:
#head(validation_ready)

In [4]:
apply_hmf_mods <- function (df, build, model, features) 
{
    model_features <- models[[features]]
    mod_lr <- hmf_models[[build]][[features]][[model]]["mod_lr"]$mod_lr
    mod_os <- hmf_models[[build]][[features]][[model]]["mod_os"]$mod_os
    if (nrow(df) > 0) {
        X <- as.matrix(df %>% select(all_of(model_features)))
        get_preds_X(X, df %>% pull(patient_id), mod_lr, mod_os)
    }
}

### 1 - Apply models

In [5]:
models <- list( 
    "tmb_bin" = c("tmb_bin"),
    "base_bin" = c("tmb_bin", "pdl1"),
    "tmb" = c("tmb"),
    "base" = c("tmb", "pdl1"),
    "rna" = c("tcell", "prolif", "tgfb"),
    #"no_pretreat" = c("tcell", "prolif", "tgfb", "tmb"),
    "no_tmb" = c("pretreat", "tcell", "prolif", "tgfb"),
    "five_latent" = c("tmb", "tcell", "prolif", "tgfb", "pretreat"),
    "five_latent_purity" = c("tmb", "tcell", "prolif", "tgfb", "pretreat", "purity")
)

In [6]:
preds <- data.frame()
for( i in names(hmf_models)){
    for (j in c("all", "bladder", "lung", "skin", "other")){
        print(j); #flush.console()
        for( k in names(models)){
            if( k != 'five_latent_hmf' ){
                df_go <- validation_ready %>% filter(model_apply == j)
                preds_i <- apply_hmf_mods( df_go, build = i, model = j, features = k )
                preds_i$model <- k
                preds_i$build <- i
                preds <- bind_rows(preds, preds_i)
            }
        }
    }
}
preds <- preds %>% unique() ### remove duplicates

[1] "all"
[1] "bladder"
[1] "lung"
[1] "skin"
[1] "other"


In [7]:
preds_mean <- preds %>% group_by(patient_id, model) %>% summarise_all( ~mean(.x, na.rm = TRUE)) %>% ungroup()

Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: ret

“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm

“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm

“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm

“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm

...

“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm

“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm

“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm

“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm

“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”Warning message in mean.default(.x, na.rm

### 2 - Ok 

In [ ]:
validation_go <- (
    validation_ready 
        %>% inner_join(preds_mean, by = "patient_id")
        %>% mutate(
            tcell = as.numeric(tcell),
            prolif = as.numeric(prolif),
            tgfb = as.numeric(tgfb), 
            pdl1 = as.numeric(pdl1), 
            pretreat = as.numeric(pretreat),
            purity = as.numeric(purity),
            pretreat_comp = as.numeric(pretreat_comp),
            age = as.numeric(age))
)

### 3 - add prediction groups

In [ ]:
validation_go$pred_os2 <- ifelse( validation_go$pred_os > 3, 3.1, validation_go$pred_os)
validation_go$lr_gp <-  cut( validation_go$pred_lr,  breaks = c(0,.1,.5,    1) , labels = c("Low", "Medium", "High"))
validation_go$os_gp <-  cut( validation_go$pred_os2,  breaks = c(0,.5,1.5,   20) , labels = c("Low", "Medium", "High"))

### 4 - send it

In [ ]:
saveRDS(validation_go, file = paste0(TMP_DIR,"validation-measure.Rds"))